In [1]:
import urllib
import urllib.request
import urllib.parse # 위 3개는 url 처리
import bs4          # html parsing  
import re           # 정규표현식 패턴 탐색 (regular expression)
import os           # 파일생성시 파일의 존재유무 확인용
import time
from concurrent.futures import ThreadPoolExecutor # 멀티쓰레드 활용, 병렬로 처리
 
 
def deleteTag(x):
    return re.sub("<[^>]*>", "", x)
 
def getComments(code):
    def makeArgs(code, page):
        params = {
            'code': code,
            'type': 'after',
            'isActualPointWriteExecute': 'false',
            'isMileageSubscriptionAlready': 'false',
            'isMileageSubscriptionReject': 'false',
            'page': page
        }
        return urllib.parse.urlencode(params)
        # code=1000&...&...&...&...&page=page
 
    def innerHTML(s, sl=0):
        ret = ''
        for i in s.contents[sl:]:
            if i is str:
                ret += i.strip() # 양쪽 공백 제거
            else:
                ret += str(i)
        return ret
 
    def fText(s):
        if len(s): return innerHTML(s[0]).strip()
        return ''
    
     # getComments 시작
    retList = []
    colSet = set() # set 선언 # dict의 key만 있는 형태, 중복 불가
    print("Processing: %d" % code) # 이번에 실행되는 naver영화의 code번호 알려주기
    #190325 공룡왕 디노 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&target=after&sword=190325&page=1
    page = 1 # page number 영화 리뷰페이지 (page number당 10개 review)
    while 1:
        try:
            f = urllib.request.urlopen(
                "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?" + makeArgs(code, page))
                #"https://movie.naver.com/movie/point/af/list.nhn?" + makeArgs(code, page))
            #url_ = "https://movie.naver.com/movie/point/af/list.nhn?" + makeArgs(code, page)
            url_ = "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?" + makeArgs(code, page)
            print(url_)
            data = f.read().decode('utf-8')
        except:
            break
            
#         soup = bs4.BeautifulSoup(data, "html.parser")
#         print(soup)
        soup = bs4.BeautifulSoup(re.sub("&#(?![0-9])", "", data), "html.parser")
        # re.sub  => &#문자 뒤에 숫자가 아닌 것들이 들어가 있는 것들만 뽑아낸다. <= 정규표현식 negative lokkahead
        # 정규표현식 가독성 https://regexper.com/
        
        cs = soup.select(".score_result li")
        # select로 특정 class의 내용을 뽑아오는데, 리뷰를 담고 있는 li들만 뽑아온다, 
        # socre_result class가 10개 리뷰를 담고 있는 class
        # class이므로 .으로 접근, li까지 접근 # id는 #으로 접근 # class나 id이름에 띄어쓰기 있으면 .으로 표현
        # print(cs) => 리뷰 10개, 한 page당 10개 리뷰 확인
        # cs[0], cs[1], ,,, cs[9] 
            
            
        if not len(cs): break # len(cs) = 10
        for link in cs: # cs[0], cs[1], ,,, cs[9] 
            try:
                url = link.select('.score_reple em a')[0].get('onclick')
                # class score_reple em a의 내용중 onclick이 가르키는 문장 가져옴 # score_reple에 리뷰 있음
                
            except:
                print(page)
                print(data)
                raise ""
            m = re.search('[0-9]+', url) # [0-9]사이의 숫자가 1번이상 나오는 것 찾아서 # match(), search(), findall(), finditer()
            if m:
                url = m.group(0) # 리뷰를 적은 각 개인과 관계있는 7자리 번호를 저장
            else:
                url = ''
            if url in colSet: return retList # colSet안에 url과 동일한 값이 있는지 확인, 있으면 retList리턴한다ㅏ. 
            colSet.add(url)
            cat = fText(link.select('.star_score em'))   # 평점을 가져온다
            cont = fText(link.select('.score_reple p'))  # 리뷰를 가져온다.
            #print(cont)
            cont = re.sub('<span [^>]+>.+?</span>', '', cont) # (complier, 대체문자, 대체대상) # <span> tag제거
            #print(cont)
            retList.append((url, cat, cont))
        page += 1
 
    return retList
 
# main 역할 함수, txt파일에 리뷰내용 저장한다.  
def fetch(i): # i 파일이름 => movie code 사용할 예정
    outname = 'comments/%d.txt' % i
    try:
        if os.stat(outname).st_size > 0: return # 파일이름이 있으면 리턴
    except:
        None
    rs = getComments(i) # 파일이름으로 movie code  사용 # 리턴 [(url(개인ID), 평점, 리뷰), , , ,]
    if not len(rs): return
    f = open(outname, 'w', encoding='utf-8')
    f.write('INSERT IGNORE INTO movie VALUES ')
    for idx, r in enumerate(rs):
        if idx: f.write(',\n')
        f.write("(%d,%s,%s,'%s')" % (i, r[0], r[1], r[2].replace("'", "''").replace("\\", "\\\\"))) # (moviecode, 개인ID, 평점, 리뷰)
    f.write(';\n')
    f.close()
    time.sleep(1)

# 병렬 멀티쓰레드 처리
movie_code = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for i in range(150000, 150500): #movie code 
        executor.submit(fetch, i)

Processing: 150001Processing: 150000

Processing: 150002
Processing: 150003
Processing: 150004
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150002&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150001&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150003&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150004&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150000&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&i

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150045&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
Processing: 150050
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150046&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150047&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
Processing: 150051
Processing: 150052
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150048&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150049&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150071&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=4
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150083&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150081&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
Processing: 150085http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150082&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
Processing: 150086

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150084&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
Processing: 15008

Processing: 150103
Processing: 150104
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150102&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150103&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
Processing: 150105
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150104&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1
Processing: 150106http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150097&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=5
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150091&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscription

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150091&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=14
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150126&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150117&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=5
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150097&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=12
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150127&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2
Processing: 150128
http://movie.naver.com/movie/bi/mi

...

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2930
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2931
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2932
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2933
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2934
http://movie.naver.com/movie/bi/mi/point

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2973
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2974
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2975
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2976
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=2977
http://movie.naver.com/movie/bi/mi/point

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3016
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3017
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3018
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3019
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3020
http://movie.naver.com/movie/bi/mi/point

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3059
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3060
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3061
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3062
http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3063
http://movie.naver.com/movie/bi/mi/point

http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=150198&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=3102


In [ ]:
import urllib
import urllib.request
import urllib.parse # 위 3개는 url 처리
import bs4          # html parsing  
import re           # 정규표현식 패턴 탐색 (regular expression)
import os           # 파일생성시 파일의 존재유무 확인용
import time
from concurrent.futures import ThreadPoolExecutor # 멀티쓰레드 활용, 병렬로 처리
 
MOVIEINDEXFILE ='movie_index.txt'    
BASEURL     = 'http://movie.naver.com/movie/point/af/list.nhn'
RATINGURL   = BASEURL + '?&page=%s' # 평점
MOVIEURL    = BASEURL + '?st=mcode&target=after&sword=%s&page=%s' # %s 129406 미쓰 와이프 
    
def getMovieIndex(filename): # MOVIEINDEXFILE ='movie_index.txt'  
    try:
        if os.path.exists(filename):
            # print(filename)
            # import pdb; pdb.set_trace()
            with open(filename, 'r') as f:
                movie_id, name = map(int,f.read().split('\n')[0].split(','))
            # movie_id, total = map(int, utils.read_txt(filename).split('\n')[0].split(','))
            # read() 전체, readline() 한줄 str # readlines() 전체 list # split() list
        else:
            movie_id, total = 190325, 0 # 129406 미쓰 와이프 # 174294 성혜의 나라 # 190325 공룡왕 디노
    print(movie_id, total)
    except:
        print("movie_index file error")
                               
    
    return [movie_id, name]  # 129406,0

def getMovie():


def getComments(code):
    def makeArgs(code, page):
        params = {
            'code': code,
            'type': 'after',
            'isActualPointWriteExecute': 'false',
            'isMileageSubscriptionAlready': 'false',
            'isMileageSubscriptionReject': 'false',
            'page': page
        }
        return urllib.parse.urlencode(params)
        # code=1000&...&...&...&...&page=page
 
    def innerHTML(s, sl=0):
        ret = ''
        for i in s.contents[sl:]:
            if i is str:
                ret += i.strip() # 양쪽 공백 제거
            else:
                ret += str(i)
        return ret
 
    def fText(s):
        if len(s): return innerHTML(s[0]).strip()
        return ''
    
     # getComments 시작
    retList = []
    colSet = set() # set 선언 # dict의 key만 있는 형태, 중복 불가
    print("Processing: %d" % code) # 이번에 실행되는 naver영화의 code번호 알려주기
    #190325 공룡왕 디노 https://movie.naver.com/movie/point/af/list.nhn?st=mcode&target=after&sword=190325&page=1
    page = 1 # page number 영화 리뷰페이지 (page number당 10개 review)
    while 1:
        try:
            f = urllib.request.urlopen(
                "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?" + makeArgs(code, page))
                #"https://movie.naver.com/movie/point/af/list.nhn?" + makeArgs(code, page))
            #url_ = "https://movie.naver.com/movie/point/af/list.nhn?" + makeArgs(code, page)
            url_ = "http://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?" + makeArgs(code, page)
            print(url_)
            data = f.read().decode('utf-8')
        except:
            break
            
#         soup = bs4.BeautifulSoup(data, "html.parser")
#         print(soup)
        soup = bs4.BeautifulSoup(re.sub("&#(?![0-9])", "", data), "html.parser")
        # re.sub  => &#문자 뒤에 숫자가 아닌 것들이 들어가 있는 것들만 뽑아낸다. <= 정규표현식 negative lokkahead
        # 정규표현식 가독성 https://regexper.com/
        
        cs = soup.select(".score_result li")
        # select로 특정 class의 내용을 뽑아오는데, 리뷰를 담고 있는 li들만 뽑아온다, 
        # socre_result class가 10개 리뷰를 담고 있는 class
        # class이므로 .으로 접근, li까지 접근 # id는 #으로 접근 # class나 id이름에 띄어쓰기 있으면 .으로 표현
        # print(cs) => 리뷰 10개, 한 page당 10개 리뷰 확인
        # cs[0], cs[1], ,,, cs[9] 
            
            
        if not len(cs): break # len(cs) = 10
        for link in cs: # cs[0], cs[1], ,,, cs[9] 
            try:
                url = link.select('.score_reple em a')[0].get('onclick')
                # class score_reple em a의 내용중 onclick이 가르키는 문장 가져옴 # score_reple에 리뷰 있음
                
            except:
                print(page)
                print(data)
                raise ""
            m = re.search('[0-9]+', url) # [0-9]사이의 숫자가 1번이상 나오는 것 찾아서 # match(), search(), findall(), finditer()
            if m:
                url = m.group(0) # 리뷰를 적은 각 개인과 관계있는 7자리 번호를 저장
            else:
                url = ''
            if url in colSet: return retList # colSet안에 url과 동일한 값이 있는지 확인, 있으면 retList리턴한다ㅏ. 
            colSet.add(url)
            cat = fText(link.select('.star_score em'))   # 평점을 가져온다
            cont = fText(link.select('.score_reple p'))  # 리뷰를 가져온다.
            #print(cont)
            cont = re.sub('<span [^>]+>.+?</span>', '', cont) # (complier, 대체문자, 대체대상) # <span> tag제거
            #print(cont)
            retList.append((url, cat, cont))
        page += 1
 
    return retList
 
# main 역할 함수, txt파일에 리뷰내용 저장한다.  
def fetch(i): # i 파일이름 => movie code 사용할 예정
    outname = 'comments/%d.txt' % i
    try:
        if os.stat(outname).st_size > 0: return # 파일이름이 있으면 리턴
    except:
        None
    rs = getComments(i) # 파일이름으로 movie code  사용 # 리턴 [(url(개인ID), 평점, 리뷰), , , ,]
    if not len(rs): return
    f = open(outname, 'w', encoding='utf-8')
    f.write('INSERT IGNORE INTO movie VALUES ')
    for idx, r in enumerate(rs):
        if idx: f.write(',\n')
        f.write("(%d,%s,%s,'%s')" % (i, r[0], r[1], r[2].replace("'", "''").replace("\\", "\\\\"))) # (moviecode, 개인ID, 평점, 리뷰)
    f.write(';\n')
    f.close()
    time.sleep(1)

# 병렬 멀티쓰레드 처리
movie_code = []
with ThreadPoolExecutor(max_workers=5) as executor:
    for i in range(150000, 150500): #movie code 
        executor.submit(fetch, i)